In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('AirlineData.csv')

In [ ]:
df.head()

In [7]:
pip install tensorflow 

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.0/227.0 MB 43.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 73.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 54.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 63.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 68.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [17]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

# Load and preprocess data
df = pd.read_csv('AirlineData.csv')  # Update the path if needed

# Preprocessing steps
df['Date'] = pd.to_datetime(df['Date'])
df['Departure_24hr'] = pd.to_datetime(df['Departure_24hr'], format='%H:%M').dt.time
df['Arrival_24hr'] = pd.to_datetime(df['Arrival_24hr'], format='%H:%M').dt.time

df['Day_of_Week'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month

# Extract hour and minute separately
df['Departure_Hour'] = df['Departure_24hr'].apply(lambda x: x.hour)
df['Departure_Minute'] = df['Departure_24hr'].apply(lambda x: x.minute)
df['Arrival_Hour'] = df['Arrival_24hr'].apply(lambda x: x.hour)
df['Arrival_Minute'] = df['Arrival_24hr'].apply(lambda x: x.minute)

# Optionally use sine and cosine transformations to encode cyclic patterns
df['Departure_Hour_sin'] = np.sin(2 * np.pi * df['Departure_Hour'] / 24)
df['Departure_Hour_cos'] = np.cos(2 * np.pi * df['Departure_Hour'] / 24)
df['Arrival_Hour_sin'] = np.sin(2 * np.pi * df['Arrival_Hour'] / 24)
df['Arrival_Hour_cos'] = np.cos(2 * np.pi * df['Arrival_Hour'] / 24)

# Drop the original time columns
df.drop(columns=['Departure_24hr', 'Arrival_24hr'], inplace=True)

# One-hot encoding for categorical columns
categorical_columns = ['Airline', 'Source', 'Destination', 'Class']
df = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Define features and target
features = df.drop(columns=['price in CAD', 'Date'])
target = df['price in CAD']

# Scaling features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# Define the ANN model
ann_model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1)  # Output layer for regression
])

# Compile the ANN model
ann_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the ANN model
history_ann = ann_model.fit(X_train, y_train, epochs=20, batch_size=64, validation_split=0.2, verbose=1)

# Evaluate the ANN model
ann_eval = ann_model.evaluate(X_test, y_test)
print(f'ANN Evaluation: {ann_eval}')

# Predict on the test data
y_pred = ann_model.predict(X_test)

# Compute R^2
r2 = r2_score(y_test, y_pred)
print(f'R^2 score: {r2:.4f}')


Epoch 1/20
3847/3847 ━━━━━━━━━━━━━━━━━━━━ 1s 301us/step - loss: 6329136.0000 - mae: 1796.9226 - val_loss: 1916473.8750 - val_mae: 901.0441
Epoch 2/20
3847/3847 ━━━━━━━━━━━━━━━━━━━━ 1s 284us/step - loss: 2029461.7500 - mae: 945.6212 - val_loss: 1719872.5000 - val_mae: 846.8181
Epoch 3/20
3847/3847 ━━━━━━━━━━━━━━━━━━━━ 1s 279us/step - loss: 1891535.3750 - mae: 908.4332 - val_loss: 1650883.2500 - val_mae: 820.7174
Epoch 4/20
3847/3847 ━━━━━━━━━━━━━━━━━━━━ 1s 279us/step - loss: 1816153.1250 - mae: 890.3162 - val_loss: 1607540.6250 - val_mae: 809.0639
Epoch 5/20
3847/3847 ━━━━━━━━━━━━━━━━━━━━ 1s 279us/step - loss: 1809435.8750 - mae: 882.6962 - val_loss: 1582479.1250 - val_mae: 795.1128
Epoch 6/20
3847/3847 ━━━━━━━━━━━━━━━━━━━━ 1s 278us/step - loss: 1776497.1250 - mae: 874.2489 - val_loss: 1567624.7500 - val_mae: 782.9029
Epoch 7/20
3847/3847 ━━━━━━━━━━━━━━━━━━━━ 1s 295us/step - loss: 1772764.7500 - mae: 868.7355 - val_loss: 1536914.2500 - val_mae: 778.0770
Epoch 8/20
3847/3847 ━━━━━━━━━━━━